In [325]:
from math import log2

#wczytanie danych
with open('cw 1/test2.txt', 'r') as f:
    element = [line.strip() for line in f]

lista = [el.split(',') for el in element]

In [326]:
lista

[['0', '0', '0', '0', '1'],
 ['0', '1', '0', '2', '1'],
 ['1', '0', '1', '2', '1'],
 ['1', '0', '1', '1', '1'],
 ['1', '0', '0', '2', '1'],
 ['0', '1', '0', '0', '0'],
 ['1', '1', '0', '0', '0'],
 ['0', '0', '1', '1', '0'],
 ['1', '1', '1', '0', '0'],
 ['1', '0', '1', '0', '0']]

In [327]:
liczbaAtrybutów = len(lista[0])-1
liczbaAtrybutów

4

In [328]:
#lista atrybutów (można ew jak wczytac z pliku)
labels = ["A"+str(nr+1) for nr in range(len(lista[0])-1)]
labels

['A1', 'A2', 'A3', 'A4']

In [329]:
liczbaWierszy = len(lista)
liczbaWierszy

10

In [330]:
#ostatnia kolumna - decyzje
decyzje = [wiersz[liczbaAtrybutów] for wiersz in lista]
decyzje

['1', '1', '1', '1', '1', '0', '0', '0', '0', '0']

In [331]:
def decyzjeKolumna(dane):
    return [kol[-1] for kol in dane]

In [332]:
decyzjeKolumna(lista)

['1', '1', '1', '1', '1', '0', '0', '0', '0', '0']

In [333]:
#dla każdej unikalnej wartosci w kolumnie zliczam wystąpienia
def licznikUnikalnychWartosci(nrKolumny,dane):
    unikalne = {}
    for wiersz in dane:
        wartosc = wiersz[nrKolumny]
        if wartosc not in unikalne:
            unikalne[wartosc] = 0
        unikalne[wartosc] +=1
    
    return unikalne

In [334]:
for nrK in range(liczbaAtrybutów):
    print(licznikUnikalnychWartosci(nrK,lista))

{'0': 4, '1': 6}
{'0': 6, '1': 4}
{'0': 5, '1': 5}
{'0': 5, '2': 3, '1': 2}


In [335]:
#ile jest unikalnych wartosci w kolumnie
def ileWartWKolumnie(nrKolumny):
    klucze = list(licznikUnikalnychWartosci(nrKolumny,lista).keys())
    licznik = len(klucze)
    
    return licznik

In [336]:
for nrK in range(liczbaAtrybutów):
    s1 ={}
    s1[nrK] = ileWartWKolumnie(nrK)
    print(s1)

{0: 2}
{1: 2}
{2: 2}
{3: 3}


In [337]:
#jakie sa unikalne wartosci w kolumnie
def unikalneWartosciWKolumnie(nrKolumny,dane):
    kolumna = [wiersz[nrKolumny] for wiersz in dane]
    unikalne = set([wartosc for wartosc in kolumna])
    return list(unikalne)

In [338]:
for nrK in range(liczbaAtrybutów):
    print(unikalneWartosciWKolumnie(nrK,lista))

['1', '0']
['1', '0']
['1', '0']
['1', '2', '0']


In [339]:
# tworzy słownik z unikalnymi wartosciami i padajacymi po nich decyzjami
def wartosciDecyzji(k):   # k to kolumna
    dlaKolumny = {}
    for ipk,pk in enumerate(k):
        tab = [decyzje[iel] for iel,el in enumerate(k) if el==pk ] #jakie sa decyzje dla wartosci
        #print(tab)
        dlaKolumny[pk] = {wart : tab.count(wart) for wart in decyzje}
    return dlaKolumny

In [340]:
k = [wiersz[1] for wiersz in lista]
k

['0', '1', '0', '0', '0', '1', '1', '0', '1', '0']

In [341]:
wartosciDecyzji(k)

{'0': {'1': 4, '0': 2}, '1': {'1': 1, '0': 3}}

In [342]:
Info = {}
Gain = {}
Split = {}
GainRatio = {}

maxA=0
T= liczbaWierszy

Entropia

In [343]:
#entropia
def I(P):
    wynik = (-1)*sum([p*log2(p) for p in P if p!=0])
    return wynik

In [344]:
podsumowanieDecyzji = licznikUnikalnychWartosci(liczbaAtrybutów,lista) # unikalne decyzje+licznik
print(podsumowanieDecyzji)
zbior = [podsumowanieDecyzji[a]/len(decyzje) for a in podsumowanieDecyzji] #prawdopodobieństo dla całego zbioru
print(zbior)

{'1': 5, '0': 5}
[0.5, 0.5]


In [345]:
def Informacje(numer):
    info=0
    split=0
    print("\n")
    print(labels[numer]) #atrybut
    print(licznikUnikalnychWartosci(numer,lista)) 
    print(liczbaWierszy)
   
    #for w in range(len(licznikUnikalnychWartosci(numer))):
    k = [wiersz[numer] for wiersz in lista] #cala kolumna
    for wart in list(licznikUnikalnychWartosci(numer,lista).keys()):
        licznikDecyzji = wartosciDecyzji(k)        
        print("\n")
        print("\t",'Decyzje:')
        print("\t",wart)
        print("\t",licznikDecyzji[wart]) 
        liczebnosc = [licznikDecyzji[wart][a] for a in licznikDecyzji[wart]]
        Ti = sum(liczebnosc)
        PP = [licznikDecyzji[wart][a]/Ti if Ti!=0 else "0" for a in licznikDecyzji[wart]]
        print("\t",PP)
        print("\t",I(PP),'*', Ti/T)
        
        info += I(PP) * (Ti/T)
        split += ((-1)*(Ti/T)* log2(Ti/T))

        print("\t", info)
        
    gain = I(zbior)-info
    if split > 0:
        gainRatio = gain/split
    
    Info[numer] = info
    Gain[numer] = gain
    Split[numer] = split
    GainRatio[numer] = gainRatio

In [346]:
print("=====")
print(podsumowanieDecyzji)
print(zbior)
print("Entropia:",I(zbior)) # etropia dla całego zbioru
print("=====")

for nr in range(liczbaAtrybutów):  
    Informacje(nr)

=====
{'1': 5, '0': 5}
[0.5, 0.5]
Entropia: 1.0
=====


A1
{'0': 4, '1': 6}
10


	 Decyzje:
	 0
	 {'1': 2, '0': 2}
	 [0.5, 0.5]
	 1.0 * 0.4
	 0.4


	 Decyzje:
	 1
	 {'1': 3, '0': 3}
	 [0.5, 0.5]
	 1.0 * 0.6
	 1.0


A2
{'0': 6, '1': 4}
10


	 Decyzje:
	 0
	 {'1': 4, '0': 2}
	 [0.6666666666666666, 0.3333333333333333]
	 0.9182958340544896 * 0.6
	 0.5509775004326937


	 Decyzje:
	 1
	 {'1': 1, '0': 3}
	 [0.25, 0.75]
	 0.8112781244591328 * 0.4
	 0.8754887502163469


A3
{'0': 5, '1': 5}
10


	 Decyzje:
	 0
	 {'1': 3, '0': 2}
	 [0.6, 0.4]
	 0.9709505944546686 * 0.5
	 0.4854752972273343


	 Decyzje:
	 1
	 {'1': 2, '0': 3}
	 [0.4, 0.6]
	 0.9709505944546686 * 0.5
	 0.9709505944546686


A4
{'0': 5, '2': 3, '1': 2}
10


	 Decyzje:
	 0
	 {'1': 1, '0': 4}
	 [0.2, 0.8]
	 0.7219280948873623 * 0.5
	 0.36096404744368116


	 Decyzje:
	 2
	 {'1': 3, '0': 0}
	 [1.0, 0.0]
	 -0.0 * 0.3
	 0.36096404744368116


	 Decyzje:
	 1
	 {'1': 1, '0': 1}
	 [0.5, 0.5]
	 1.0 * 0.2
	 0.5609640474436812


In [347]:
print("\n")
for nr in range(liczbaAtrybutów):
    print(f"Info({labels[nr]},T)= {Info[nr]}   \t Gain({labels[nr]},T)= {Gain[nr]}")

#wyznacza max Gain
for a in Gain:
    if Gain[a]==max(Gain.values()):
        maxA=a

print(f"\nWybrany atrybut {labels[maxA]}, bo w tym przypadku najwyższa jest wartość Gain: {max(Gain.values())}")

print("\n")
for nr in range(liczbaAtrybutów): 
    print(f"Split(D{str(nr+1)},T)= {Split[nr]}   \t Gain Ratio(D{str(nr+1)},T)= {GainRatio[nr]}")



Info(A1,T)= 1.0   	 Gain(A1,T)= 0.0
Info(A2,T)= 0.8754887502163469   	 Gain(A2,T)= 0.12451124978365313
Info(A3,T)= 0.9709505944546686   	 Gain(A3,T)= 0.02904940554533142
Info(A4,T)= 0.5609640474436812   	 Gain(A4,T)= 0.4390359525563188

Wybrany atrybut A4, bo w tym przypadku najwyższa jest wartość Gain: 0.4390359525563188


Split(D1,T)= 0.9709505944546686   	 Gain Ratio(D1,T)= 0.0
Split(D2,T)= 0.9709505944546686   	 Gain Ratio(D2,T)= 0.12823644219877584
Split(D3,T)= 1.0   	 Gain Ratio(D3,T)= 0.02904940554533142
Split(D4,T)= 1.4854752972273344   	 Gain Ratio(D4,T)= 0.29555250994465343


In [348]:
listaNumerow = [nr for nr in range(len(lista[0])-1)]
listaNumerow

[0, 1, 2, 3]

In [349]:
def maxAtryb(lista, atryb):
    #maxA=0
    if atryb in lista:  
        del lista[atryb]
    g = [Gain[a] for a in lista]
    maxGain=max(g)

    for ga in Gain:
        if Gain[ga] == maxGain:
            maxAtrybut=ga
            
    return maxAtrybut, lista

        
# def nowaLista(dane, nrWiersza):
#     nowaLista(dane, atryb, w):
#     nowaL = []
#     nowaL.append(dane[nrWiersza])
#     return nowaL

def nowaLista(dane, atryb, value):
    newSet = []
    for wiersz in dane:
        if wiersz[atryb] == value:
            newSetSample = wiersz[:atryb]        #making copy
            newSetSample.extend(wiersz[atryb:])#extends the list by adding all items of a list
            newSet.append(newSetSample)     #add to new Set
    return newSet

def class_counts(rows):
    counts = {}
    for row in rows:
        if row not in counts.keys():
            counts[row] = 0
        counts[row] += 1
    return max(counts)

In [350]:
import json
drzewo = {}

def szukaj(dane, l, atryb):
    #print("\nNajlepszy atrybut: ", labels[atryb])
    
    d = decyzjeKolumna(dane)

    #wyswietla decyzje, koniec podziału
    if d.count(d[0]) == len(d):
        print(d[0])
        
    if len(dane[0]) == 1:
        return class_counts(d)
    
        
    atrybut = labels[atryb]
    drzewo = { atrybut: {} }
#     del(l[atryb])
    unik = unikalneWartosciWKolumnie(atryb,dane)

    print(drzewo)
    for w in unik:
        #listaA = l[:] #kopia listy  numerów  
        noweA, listaA = maxAtryb(l, atryb) #szuka max w podaje liscie
        drzewo[atrybut][w] = szukaj(nowaLista(dane, atryb, w), listaA, noweA)
        print(drzewo)

        
    
        
    return drzewo
    
    
    #for wart in unikalneWartosciWKolumnie(atryb,lista):
        
        #print(f"{tab}{labels[atryb]} = {wart}")
             # ,unikalne[atryb][w],":", [a for a in C[w] if C[w][a]!=0]) 
  
                        
    
def budujDrzewo():
    print("\nDrzwo decyzyjne dla {0} przypadków ({1} atrybutów)".format(liczbaWierszy,liczbaAtrybutów))
    szukaj(lista, listaNumerow, maxA) 
    
koniec = budujDrzewo()
print(json.dumps(koniec, indent=4, sort_keys=True))


Drzwo decyzyjne dla 10 przypadków (4 atrybutów)
{'A4': {}}


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
print(json.dumps(koniec, indent=4, sort_keys=True))

In [ ]:
drzewo


In [ ]:
#     k = [wiersz[atryb] for wiersz in dane] 
#     #print(k)
#     #drzewo[atryb] = wartosciDecyzji(k)
#     #print(wartosciDecyzji(k))
#     for nrWiersza,a in enumerate(wartosciDecyzji(k)):
#         print(f"{tab}{labels[atryb]} = {a}")
#         for w in wartosciDecyzji(k)[a]:
#             if wartosciDecyzji(k)[a][w] != 0:

#                 #nowaLista(dane, nrWiersza)
#                 #nowaLista.append(dane[nrWiersza])
#                 print("działam")
#             elif wartosciDecyzji(k)[a][w] == 0:
#                 print(f"\t{tab}: {w}")
# #                 print(nrWiersza)
#                 del dane[nrWiersza]
#             #print(dane)
#             noweA = maxAtryb(atryb)
#             if noweA == None:
#                 continue
#             szukaj(noweA,dane)

#tab="\t"*atryb